In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import unicodedata
import re
import json

# TOKENİZER - TENSORFLOW

------

## 🔤 Tokenizer Nedir?

**Tokenizer**, ham metni modelin anlayabileceği sayısal dizilere dönüştüren araçtır. Her kelimeye (veya karaktere) bir ID atar.



### 🎯 Temel Görevi
- Metni bölmek (tokenize etmek)
- Her tokena bir numara vermek (vocab oluşturmak)
- Bu numaralara göre diziler üretmek



### ⚙️ Kullanım Parametreleri

| Parametre     | Açıklama |
|---------------|----------|
| `oov_token`   | Eğitimde görülmeyen kelimeler için özel token (`<OOV>`) |
| `filters`     | Hangi karakterlerin silineceği (örn. noktalama) |
| `char_level`  | `True`: karakter bazlı, `False`: kelime bazlı tokenize |



### 🧱 Adımlar

1. **Tokenizer oluştur**  
   ```python
   tokenizer = Tokenizer(oov_token="<OOV>", filters="", char_level=False)


-------

## 🔑 SOS ve EOS Tokenları Nedir?

**SOS (Start of Sequence)** ve **EOS (End of Sequence)** tokenları, dizilerle çalışan modellerde giriş ve çıkış dizilerinin sınırlarını belirlemek için kullanılan özel işaretleyicilerdir.



### 🎯 Amaçları
- **sos**: Modelin dizinin **başladığını** anlamasını sağlar.
- **eos**: Modelin dizinin **bittiğini** anlamasını sağlar.



### 📦 Kullanım Alanları
- **Seq2Seq modellerinde** (örneğin: encoder-decoder yapıları)
- **Makine çevirisi**
- **Dil modeli eğitimi ve tahmini**
- **Beam Search gibi sıralı üretim algoritmalarında** üretimin ne zaman duracağını belirlemek için



### 🧱 Tipik Uygulama Şekli
**Hedef metinler** şu şekilde hazırlanır:

- `target_input`: `sos` + gerçek cümle  
- `target_output`: gerçek cümle + `eos`

Bu sayede model:
- Ne zaman üretmeye başlayacağını (sos),
- Ne zaman durması gerektiğini (eos) öğrenmiş olur.



### 🧠 Örnek
Gerçek cümle: `Merhaba dünya`

- `target_input`: `sos Merhaba dünya`  
- `target_output`: `Merhaba dünya eos`


### 💡 Not
- Bu tokenlar, tokenizer'a *manüel* olarak metnin bir parçası gibi eklenmelidir.
- Tokenizer, onları kelime listesine dahil ederek ID atar (örneğin: `sos → 2`).





In [2]:
df = pd.read_csv("örnek_set.csv")
df.head()

,input,output
0,Merhaba,"Merhaba, size nasıl yardımcı olabilirim?"
1,Nasılsın?,"İyiyim, teşekkür ederim. Siz nasılsınız?"
2,Adın ne?,Ben bir yapay zekâ asistanıyım. Adım yok ama y...
3,Kaç yaşındasın?,"Benim yaşım yok, dijitalim!"
4,Bugün günlerden ne?,"Maalesef tarih bilgim yok, ama sistem saatinde..."


In [3]:
input_texts = df['input'].astype(str).to_list()
target_texts = df['output'].astype(str).to_list()

In [4]:
token_in = ['<start> ' + text for text in target_texts]
token_out = [text + ' <end>' for text in target_texts]

input_token = Tokenizer(filters='' , oov_token='<OOV>')
input_token.fit_on_texts(input_texts)

target_token = Tokenizer(filters='' , oov_token='<OOV>')
target_token.fit_on_texts(token_in + token_out)

input_seq = input_token.texts_to_sequences(input_texts)
decoder_in = target_token.texts_to_sequences(token_in)
decoder_out = target_token.texts_to_sequences(token_out)

max_len_inp = max(len(seq) for seq in input_seq)
max_len_out = max(len(seq) for seq in decoder_in + decoder_out)

max_len = max(max_len_inp , max_len_out)

encoder_input = pad_sequences(input_seq , maxlen = max_len , padding="post")
decoder_input = pad_sequences(input_seq , maxlen = max_len , padding="post")
decoder_target = pad_sequences(input_seq , maxlen = max_len , padding="post")

inp_vocab_size = len(input_token.word_index) + 1 
tar_vocab_size = len(target_token.word_index) + 1

print(inp_vocab_size)
print(tar_vocab_size)

104
249


* Yukarıda bulunan bu kod klasik bir yapı.Bizim amacımız bu yapıya daha fazla derinlik katmak.Normal tokenizer işlevlerine zaten hakimiz.Yani bu yapıyı küçük-orta ölçekli yerlerde kullanabilirsiniz.Gayet de iş görür.Ama bu notebookdaki temel amaç ön işleme adımlarını daha ciddi yerlere getirmek.Gelin bu yapılar neymiş onlara bakalım.

-----

## 📌 Neler Eklenebilir ?

### 1. `char_level`  
**Ne yapar?**  
- Tokenizer’ı kelime (`word`) yerine karakter (`char`) bazlı çalışacak şekilde ayarlar.

**Neden önemli?**  
- Çok küçük veri setlerinde kelime bazlı model **aşırı “sparse”** (seyreklik) yaşayabilir.  
- Yeni (out-of-vocab) kelimelerin tamamını `<OOV>` yerine, karakter dizisi sayesinde anlamlı parçalara ayırabilirsiniz.  
- Morfolojik açıdan zengin (örneğin Türkçe) dillerde **kök-ek ayrımı** yapmak kolaylaşır.


### 2. `max_vocab_size`  
**Ne yapar?**  
- Tokenizer’a kaç adet en sık görülen token’ı “aktif” tutacağını söylersiniz; geriye kalanlar hepsi `<OOV>` olur.

**Neden önemli?**  
- **Bellek & hız**: Gigabaytlarca sözlüğü modelde tutmak yerine, en sık kullanılan 5k–10k token’ı alıp gerisini atmak.  
- **Genelleme**: Aşırı nadir kelimeler eğitim sırasında gürültü oluşturabilir; onları `<OOV>` yapmak modelin daha sağlam öğrenmesini sağlar.


In [5]:
filters = ''
oov_token = '<OOV>'
start_token = '<start>'
end_token = '<end>'
char_level = False
max_vocab_size = 10000
padding="post"

In [6]:
token_in = [f"{start_token} {t}" for t in target_texts]
token_out = [f"{t} {end_token}" for t in target_texts]

In [7]:
input_token = Tokenizer(
    num_words=max_vocab_size,
    filters=filters,
    oov_token=oov_token,
    char_level=char_level
                        )

target_token = Tokenizer(
    num_words=max_vocab_size,
    filters=filters,
    oov_token=oov_token,
    char_level=char_level
                        )

In [8]:
input_token.fit_on_texts(input_texts)
target_token.fit_on_texts(token_in+token_out)

In [9]:
input_seq = input_token.texts_to_sequences(input_texts)
decoder_in = target_token.texts_to_sequences(token_in)
decoder_out = target_token.texts_to_sequences(token_out)

In [10]:
max_len_inp = max(len(seq) for seq in input_seq)
max_len_out = max(len(seq) for seq in decoder_in + decoder_out)
max_len = max(max_len_inp , max_len_out)
print(max_len)

12


In [11]:
encoder_input = pad_sequences(input_seq,maxlen = max_len, padding=padding)
decoder_input = pad_sequences(decoder_in,maxlen = max_len, padding=padding)
decoder_target = pad_sequences(decoder_out,maxlen = max_len, padding=padding)

In [12]:
inp_vocab_size = max(len(input_token.word_index) + 1 , max_vocab_size or float("inf"))
tar_vocab_size = max(len(target_token.word_index) + 1 , max_vocab_size or float("inf"))

In [13]:
print("Input vocab size: ", inp_vocab_size)
print("Target vocab size:", tar_vocab_size)
print(f"{start_token} token id:", target_token.word_index[start_token])
print(f"{end_token} token id:", target_token.word_index[end_token])

Input vocab size:  10000
Target vocab size: 10000
<start> token id: 2
<end> token id: 3


In [14]:
# ==== TOKENIZER SONRASI HIZLI KONTROL ====

# 1) Ham ID dizilerini direkt geri metne çevir:
sample_idx = 0  # ilk örneği incele
print("Girdi örneği (IDs→text):",
      input_token.sequences_to_texts([ input_seq[sample_idx] ])[0])
print("Decoder-in örneği:",
      target_token.sequences_to_texts([ decoder_in[sample_idx] ])[0])
print("Decoder-out örneği:",
      target_token.sequences_to_texts([ decoder_out[sample_idx] ])[0])

# 2) <start> ve <end>'i temizleyip nihai cümleyi al:
raw = target_token.sequences_to_texts([ decoder_out[sample_idx] ])[0]
clean = raw.replace(f"{start_token} ", "").replace(f" {end_token}", "").strip()
print("Temizlenmiş target:", clean)



Girdi örneği (IDs→text): merhaba
Decoder-in örneği: <start> merhaba, size nasıl yardımcı olabilirim?
Decoder-out örneği: merhaba, size nasıl yardımcı olabilirim? <end>
Temizlenmiş target: merhaba, size nasıl yardımcı olabilirim?


----
## 🚀 Profesyonel Tokenizer Pipeline İyileştirmeleri

Aşağıda “sadece çalışıp geçmek” yerine **güvenilir**, **ölçeklenebilir** ve **esnek** bir preprocessing pipeline’ı oluşturmak için başlıca adımlar özetlenmiştir.



### 1. Temizlik & Normalizasyon
- **Unicode Normalization** (`NFC`/`NFKC`):  
  Farklı biçimlerde kodlanmış karakterleri tekilleştirir.
- **Lowercasing**:  
  Tüm metni küçük harfe indirerek sözlükteki duplikasyonları önler.
- **Diakritik/Karak­ter Temizleme**:  
  Türkçe “ç, ş, ı” gibi işaretleri gerektiğinde standartlaştırır.


### 2. Özel Token’lar & Sabitleme
- **\<pad\>** token’ını açıkça tanımlayıp index’ini sıfır yapın.  
- **\<unk\>** veya **\<OOV\>** farkını belirleyin ve sabitleyin.  
- `fit_on_texts` öncesi özel token’ları manuel ekleyerek aynı ID’leri garantileyin.



### 3. Vocab Analizi & Sınırlandırma
- **max_vocab_size**: En sık kullanılan N token’ı aktif tutar, gerisi `<OOV>`.  
- **min_freq**: Belirli bir frekans altındaki token’ları da otomatik atar.  
- **Coverage Raporu**: Toplam tokenların % kaçı top-N vocab’a giriyor?  



### 4. Subword / BPE / WordPiece Desteği
- **SentencePiece** veya **HuggingFace Tokenizers** kullanarak:  
  - Byte-Pair Encoding (BPE)  
  - Unigram LM  
  - WordPiece  
- Böylece OOV sorunu en aza iner, vocab boyutu küçülür.



### 5. Parametrik & Modüler Yapı
- Expose edilecek parametreler:  
  - `lowercase: bool`  
  - `normalize_unicode: bool`  
  - `filters: str`  
  - `min_freq: int`  
  - `max_vocab_size: int`  
  - `char_level: bool`  
  - `tokenizer_backend: "keras" | "sentencepiece" | "hf"`  



### 6. Kaydetme / Versiyonlama / Geri Yükleme
- **JSON + Metadata** (parametreler, min_freq vs.) bir arada saklayın.  
- Versiyonlama: `tokenizer_v1.0.json`, `tokenizer_v1.1.json`…  
- Aynı pipeline’ı projeler arası tekrar kullanmak için.



### 7. Performans & Ölçeklenebilirlik
- **Rust-tabanlı** tokenizers (HuggingFace) ile paralel hız.  
- **Disk cache**: TFRecord/LMDB gibi formatlarda tokenize edilmiş veriyi saklama.  
- `tf.data.Dataset` içinde “cache → map → batch → prefetch” akışı.



### 8. Debug & İzlenebilirlik
- **Sample Çıktılar**: Her adımda 3–5 örnek  
- **Logging**:  
  - Toplam kelime sayısı vs. vocab boyutu  
  - Padding/truncation dağılımı  
- **Unit Test**:  
  - Edge-case cümleler  
  - Özel token ID kontrolü  
  - Truncation & padding davranışı


---
---

### 1. Lowercasing ve Unicode normalize etmek

* Basit bir clean_text() fonksiyonu yazıp, fit_on_texts çağrısından önce tüm metinleri ona geçiriyoruz.

* Bunu tokenizerların en başına ekliyoruz.

* Böylece hem bütün metinler aynı formata gelir hem tokenizer’a “gereksiz” karakterler girmez.

In [15]:

import unicodedata
import re

def clean_texts(text,lowercase=True,normalize_unicod = True):

    if normalize_unicod:
        text = unicodedata.normalize("NFC",text)

    if lowercase:
        text = text.lower()

    text = re.sub(r"[^a-zçğıöşü0-9\s]", " ", text)
    # birden fazla arka arkaya boşluğu tek boşluk yap
    text = re.sub(r"\s+", " ", text).strip()
    return text

input_texts = [clean_texts(t) for t in df["input"].astype(str)]
target_texts = [clean_texts(t) for t in df["output"].astype(str)]


---
-----

## Özel Token’lar & Sabitleme (Teori)

### Neden Özel Token’lar?
- **\<pad\>**  
  - Dizilerin sabit uzunluğa getirilmesi için kullanılan dolgu (padding) token’ı.  
  - Modelin gerçek veriden ayırt edebilmesi için özel bir ID’ye sahip olmalıdır.

- **\<unk\> / \<OOV\>**  
  - Eğitim sırasında görülmeyen veya nadir kelimeleri temsil eder.  
  - Bilinmeyen kelimelerin modelde tutarlı şekilde işlenmesini sağlar.

- **\<sos\> (Start of Sequence)**  
  - Decoder’ın diziyi nereden üretmeye başlayacağını belirtir.  
  - Sıralı üretim algoritmalarında mutlaka ilk token olarak kullanılır.

- **\<eos\> (End of Sequence)**  
  - Decoder’ın üretimi nerede durduracağını işaretler.  
  - Özellikle beam search veya greedy decoding sırasında önem taşır.



### Sabitlemenin Amacı
1. **Tutarlılık**  
   - Aynı token her çalışmada aynı ID’ye karşılık gelmeli.  
   - Özellikle eğitim ve çıkarım (inference) ortamları arasında uyumluluk sağlar.

2. **Padding Güvenliği**  
   - \<pad\> token’ının **ID = 0** olması, `pad_sequences` gibi fonksiyonların varsayılan dolgu değeriyle örtüşmesini garantiler.

3. **OOV Yönetimi**  
   - \<unk\> token’ının sabit bir ID’ye sahip olması, bilinmeyen kelimelerin daima aynı şekilde kodlanmasını sağlar ve modelin genel performansını korur.

4. **Decoder Kontrolü**  
   - \<sos\> ve \<eos\> token’larının ID’leri değişmediği sürece, dizinin başlangıç ve bitiş sınırları model tarafından kesinlikle doğru algılanır.


### Sabitleme Adımları (Özet)
- Özel token’ları içeren bir sabit eşleme (`word → ID`) tanımlanır.  
- Tokenizer başlatıldıktan hemen sonra bu sabit eşleme yüklenir, böylece özel token’lar kesin ID’lere sahip olur.  
- Ardından gerçek metin verisi ile tokenizasyon işlemi yapılır; özel token’ların ID’leri hiçbir zaman değişmez.


### Faydaları
- Modelin hem eğitim hem de çıkarım aşamasında **öngörülebilir ve tutarlı** davranmasını sağlar.  
- **Kodun bakımı** ve **yeniden üretilebilirlik** (reproducibility) açısından kritik bir uygulamadır.  
- Sıralı metin üretimi (seq2seq, Transformer) gibi ileri düzey görevlerde hata yapmayı engeller.  


## `min_freq` (Minimum Frequency) Teorisi

### Neden Gerekli?
- **Gürültü Azaltma**  
  - Veri setindeki bazı kelimeler yalnızca birkaç kez geçer; bu nadir kelimeler modelin öğrenme sürecini olumsuz etkileyebilir.  
- **Vocab Boyutunu Kontrol Etme**  
  - Büyük metin koleksiyonlarında onbinlerce farklı token olabilir.  
  - Sözlüğü küçültmek, hem bellek kullanımını hem de eğitim süresini iyileştirir.


### Nasıl Çalışır?
1. **Frekans Eşiği**  
   - Bir token’ın sözlüğe dahil edilmesi için **en az** `min_freq` kez görünmesi gerekir.
2. **Filtreleme**  
   - `min_freq = 3` ise, veri setinde 1 veya 2 kez görülen tüm token’lar  
     - Model girişinde `<unk>` (OOV) olarak işaretlenir  
     - Sözlükte yer almaz  
3. **Sonuç**  
   - Sözlükte sadece **yeterli sıklıkta** geçen token’lar kalır  
   - Nadir token’lar, modelin “garbage” öğrenmesini önler


### Faydaları
- **Daha Temiz Vocab**  
  - Gerçekten anlamlı ve istatistiksel olarak güçlü token’lar tutulur.  
- **Daha İyi Genelleme**  
  - Nadir ve gürültülü token’lar OOV’a düştüğü için model, ana kalıpları daha iyi kavrar.  
- **Kaynak Verimliliği**  
  - Küçük bir sözlük, daha az bellek ve daha hızlı matris işlemleri demektir.


### Uygulama Adımları (Teori)
1. **Ön Analiz**  
   - Tüm metinlerdeki token frekanslarını topla  
2. **Eşik Belirleme**  
   - `min_freq` değerini seç (örn. 2–5)  
3. **Token Seçimi**  
   - Sözlüğe sadece `freq ≥ min_freq` olan token’ları ekle  
4. **OOV Atama**  
   - Geri kalan tüm token’lar `<unk>` olarak işaretlenir  



**Not:** `min_freq` değeri çok yüksek ayarlanırsa önemli ama az geçen kelimeler de OOV olur; çok düşük ayarlanırsa nadir gürültüye izin verir. İdeal `min_freq`, veri setinin büyüklüğüne ve çeşitliliğine bağlı olarak deneysel belirlenir.


* Şimdi bütün bu teorik konularını diğer öğrendiğimiz yapılarla birleştirip yola devam edelim.

In [90]:
# === 1. Temizleyici Fonksiyon ===
def clean_text(text, lowercase=True, normalize_unicode=True):
    if normalize_unicode:
        text = unicodedata.normalize("NFC", text)
    if lowercase:
        text = text.lower()
    text = re.sub(r"[^a-zçı0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [123]:
# === 2. Parametreler ===
filters        = ""           # noktalama silme filtresi
oov_token      = "<unk>"
pad_token      = "<pad>"
sos_token      = "<sos>"
eos_token      = "<eos>"
char_level     = False        # True=char, False=word
max_vocab_size = None         # None = sınırsız
min_freq       = 2            # en az 2 kez geçen tokenʼlar kalacak
padding        = "post"

fixed_index = {
    pad_token : 0,
    oov_token : 1,
    sos_token : 2,
    eos_token : 3
}

In [124]:
# === 3. Veri Yükleme & Temizleme ===
df = pd.read_csv("örnek_set.csv")
raw_inputs  = df["input"].astype(str).tolist()
raw_targets = df["output"].astype(str).tolist()


In [125]:
input_texts  = [clean_text(t) for t in raw_inputs]
target_texts = [clean_text(t) for t in raw_targets]

In [126]:
dec_in_text  = [f"{sos_token} {t}" for t in target_texts]
dec_out_text = [f"{t} {eos_token}" for t in target_texts]
all_texts    = input_texts + dec_in_text + dec_out_text

In [127]:
# === 4. Geçici Tokenizer ile Frekansları Topla ===
tmp_tok = Tokenizer(
    num_words=None,
    filters=filters,
    oov_token=oov_token,
    char_level=char_level
)
tmp_tok.fit_on_texts(all_texts)
word_counts = tmp_tok.word_counts  # OrderedDict {word: count}

In [128]:
# === 5. min_freq Uygula ===
if min_freq is not None:
    keep_tokens = [w for w, cnt in word_counts.items() if cnt >= min_freq]
else:
    keep_tokens = list(word_counts.keys())

In [129]:

# === 6. max_vocab_size Uygula ===
if max_vocab_size is not None:
    keep_tokens = sorted(
        keep_tokens,
        key=lambda w: word_counts[w],
        reverse=True
    )[: max_vocab_size]


In [130]:
# === 7. Final Tokenizer Oluştur ve Fit Et ===
tokenizer = Tokenizer(
    num_words=(len(keep_tokens) + 1) if max_vocab_size is None else max_vocab_size,
    filters=filters,
    oov_token=oov_token,
    char_level=char_level
)

In [131]:
# Önce sadece keep listesiyle fit, böylece index sabitlenir
tokenizer.fit_on_texts(keep_tokens)
# Ardından tüm metinlerle fit
tokenizer.fit_on_texts(all_texts)


In [132]:
# === 8. Sequence Dönüşümü & Padding ===
inp_seq    = tokenizer.texts_to_sequences(input_texts)
dec_in_seq = tokenizer.texts_to_sequences(dec_in_text)
dec_out_seq= tokenizer.texts_to_sequences(dec_out_text)


In [133]:
maxlen = max(
    max(len(s) for s in inp_seq),
    max(len(s) for s in dec_in_seq),
    max(len(s) for s in dec_out_seq)
)


In [134]:
encoder_input  = pad_sequences(inp_seq,    maxlen=maxlen, padding=padding)
decoder_input  = pad_sequences(dec_in_seq, maxlen=maxlen, padding=padding)
decoder_target = pad_sequences(dec_out_seq,maxlen=maxlen, padding=padding)

In [135]:
vocab_size = min(len(tokenizer.word_index) + 1, max_vocab_size or float("inf"))
print("Final vocab size:   ", vocab_size)
print("Max sequence length:", maxlen)
print(f"{sos_token!r} id:", tokenizer.word_index.get(sos_token))
print(f"{eos_token!r} id:", tokenizer.word_index.get(eos_token))

Final vocab size:    329
Max sequence length: 14
'<sos>' id: 2
'<eos>' id: 3


In [ ]:
coverage = sum(word_counts[w] for w in keep_tokens) / sum(word_counts.values())
print(f"Vocabulary coverage (min_freq={min_freq}): {coverage:.2%}")

'''
Verideki her 100 token’ın yaklaşık 94.6’sı, sözlüğünüzde tuttuğunuz token’lar tarafından karşılanıyor. 
Geriye kalan ~5.4% ise nadir token’lar (1 kez geçenler) olduğu için <unk> olarak işleniyor.

Ne kadar yüksek ise modelinizin gördüğü gerçek kelimeleri ne kadar iyi kapsadığını gösterir.

Çok düşükse, belki min_freq veya max_vocab_size’ı yeniden düşünmek gerekir.

'''

Vocabulary coverage (min_freq=2): 94.61%


'\nVerideki her 100 token’ın yaklaşık 94.6’sı, sözlüğünüzde tuttuğunuz token’lar tarafından karşılanıyor. \nGeriye kalan ~5.4% ise nadir token’lar (1 kez geçenler) olduğu için <unk> olarak işleniyor.\n\n'

In [137]:
print("Top 20 tokens:")
for word, idx in list(tokenizer.word_index.items())[:20]:
    print(f"{word:<10} → {idx}")

Top 20 tokens:
<unk>      → 1
<sos>      → 2
<eos>      → 3
bir        → 4
g          → 5
ama        → 6
n          → 7
i          → 8
ben        → 9
ne         → 10
m          → 11
yardımcı   → 12
yok        → 13
de         → 14
ba         → 15
ve         → 16
nedir      → 17
r          → 18
python     → 19
ya         → 20


In [138]:
print("Sample input text:  ", tokenizer.sequences_to_texts([encoder_input[0]])[0])
print("Sample decoder-in:  ", tokenizer.sequences_to_texts([decoder_input[0]])[0])
print("Sample decoder-out: ", tokenizer.sequences_to_texts([decoder_target[0]])[0])

Sample input text:   merhaba <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
Sample decoder-in:   <sos> merhaba size nasıl yardımcı olabilirim <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
Sample decoder-out:  merhaba size nasıl yardımcı olabilirim <eos> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


---

## tf.data.Dataset Entegrasyonu (Teori)

**Nedir?**  
TensorFlow’un `tf.data.Dataset` API’si, önceden tokenize edip pad ettiğiniz NumPy dizileri veya diğer veri kaynaklarını “pipeline” olarak adlandırılan akışa dönüştürür. Bu akış üzerinde **shuffle**, **batch**, **cache**, **map** ve **prefetch** gibi işlemleri zincirleyerek, modelin eğitim sırasında veriyle kesintisiz ve verimli bir şekilde beslenmesini sağlar.


## Ne Zaman Kullanılır?
- **Büyük veri setleri** belleğe sığmadığında veya diskten “streaming” okumak gerektiğinde  
- Eğitim sırasında **shuffle** ve **batch** işlemlerini CPU/GPU ile paralel hale getirerek performansı artırmak istediğinizde  
- **Veri augmentasyonu** veya ek ön işleme adımlarını (örneğin, `map` ile token filtreleme) model akışına dahil etmek istediğinizde  
- TFRecord, CSV, görüntü dosyaları gibi formatlardan doğrudan okuma ve önbellekleme (`cache`) gerektiğinde  



## Entegrasyon Aşamaları
1. **Tokenizer & Padding**  
   - Ham metni `texts_to_sequences` ve `pad_sequences` ile NumPy dizilerine dönüştürün.  
2. **Dataset Oluşturma**  
   - `tf.data.Dataset.from_tensor_slices((inputs_dict, targets))` ile veri akışını başlatın.  
3. **Pipeline Adımları**  
   - **Shuffle:** Rastgelelik için `shuffle(buffer_size)`  
   - **Batch:** GPU/TPU’ya uygun sabit boyutlarda gruplama için `batch(batch_size)`  
   - **Prefetch:** Bir sonraki batch’i önceden yüklemek için `prefetch(tf.data.AUTOTUNE)`  
   - İsteğe bağlı: `cache()`, `repeat()` ve paralel `map()` gibi fonksiyonlar.  
4. **Model.fit**  
   - Oluşturduğunuz `dataset` objesini doğrudan `model.fit(dataset, epochs=…)` ile kullanın.  


## Nerede Yazılmalı?
- **Tokenize & pad** adımından hemen sonra,  
- **Model tanımı** ve `.compile()` çağrısından **önce**,  
- Böylece `dataset` hazır olduktan sonra doğrudan `model.fit(dataset…)` satırını yazabilirsiniz.  


* Modelin oluşumundan hemen önce ya da fit fonksiyonundan hemen önce yapmanız gerekecek.Notebook sonunda yapılacak olan projede gösterilecektir.

In [140]:
buffer_size = 1000
batch_size = 32

In [143]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((
    
    { 
      "encoder_input" : encoder_input,
      "decoder_input" : decoder_input
    },
    decoder_target
    )
)

In [ ]:
dataset = (
    dataset
    # Shuffle: veri kümesindeki örnekleri karıştırarak her epoch’ta farklı sıralama sağlar.
    # buffer_size: kaç örneğin bellekte karıştırılacağını belirler.
    #   - Büyük değer = daha random sıra, daha fazla bellek kullanımı.
    .shuffle(buffer_size=buffer_size)

    # Batch: örnekleri batch_size kadar gruplar.
    # batch_size: her adımda modele kaç örnek yollanacağını belirler.
    # drop_remainder=True: son batch’te batch_size’a tamamlanamayan
    #   eksik grup varsa, o küçük batch’i atlar.
    .batch(batch_size, drop_remainder=True)

    # Prefetch: bir sonraki batch’i eğitim çalışırken önceden hazırlar.
    # tf.data.AUTOTUNE: TensorFlow’un uygun sayıda thread seçmesini sağlar.
    .prefetch(tf.data.AUTOTUNE)
)


* Daha da ileriye taşımak istersek ; 

In [ ]:
import tensorflow as tf

# === tf.data.Dataset Entegrasyonu ===

# 1) NumPy dizilerinden Dataset oluştur:
dataset = tf.data.Dataset.from_tensor_slices(
    (
        {
            "encoder_input": encoder_input,  # Encoder’a gidecek padded giriş
            "decoder_input": decoder_input   # Decoder’a gidecek padded giriş
        },
        decoder_target  # Modelin öğreneceği gerçek çıktılar
    )
)

# 2) Pipeline adımları:
dataset = (
    dataset
    .cache()                            # 1. İlk kullanımda cache’le (bellek/disk), sonraki epoch’lar için hızlı erişim
    .shuffle(buffer_size=buffer_size)   # 2. Veriyi buffer_size kadar örnekle karıştırarak rastgelelik ekle
    .repeat()                           # 3. Dataset’i sonsuz (veya istenen sayıda) tekrar et
    .batch(batch_size, drop_remainder=True)  # 4. batch_size kadar gruplar; eksik batch’leri atar
    .prefetch(tf.data.AUTOTUNE)        # 5. Bir sonraki batch’i arka planda hazırla
)

# Artık dataset, doğrudan model.fit ile kullanılmaya hazır:
# model.fit(dataset, epochs=EPOCHS)


----

##  Morfolojik Analiz  

**Kavram:**  
Türkçe gibi eklemeli (agglutinative) dillerde, bir kelime tek bir string’de kök + çok sayıda ek barındırır.  
Morfolojik analiz, bu kelimeyi “kök” ve “ek” bileşenlerine ayırarak anlamsal yapıyı ortaya çıkarır.


### Nasıl Çalışır?  
1. **Kök Çıkarımı (Stemming/Lemmatization)**  
   - “yapabilir­-dim” → kök: “yap”, ekler: [“-abilir”, “-dim”]  
2. **Ek Sınıflandırması**  
   - Çekim ekleri (“-dim”, “-siniz”) vs. yapım ekleri (“-lük”, “-ci”)  
3. **Sözlük & Kurallar**  
   - Zemberek gibi kütüphaneler, kapsamlı morfolojik sözlük + dilbilgisi kural seti kullanır.  
4. **Çıktı**  
   - Hem kök hem de ek türleri elde edilir; örnek:  
     ```json
     {
       "surface": "yapabilir­dim",
       "lemma": ["yap"],
       "tags": ["Verb","Ability","Past","FirstPerson"]
     }
     ```

### Kullanılabilecek Araçlar  
- **Zemberek (Java)**  
  - `turkish-morphology` modülü  
  - Python için `py4j` üzerinden erişim veya `zemberek-python` sarımları  
- **TRnlp / TRmorph**  
  - Saf Python implementasyonlar  
- **SpaCy + tr_spacy**  
  - SpaCy çerçevesi içinde Türkçe morfoloji ekleri  
- **Stanza (Stanford NLP)**  
  - Çok dilli morfolojik analiz, Türkçe desteği de var


### Uygulama Örneği (Zemberek-Python)  
```python
from zemberek import TurkishMorphology

morph = TurkishMorphology.create_with_defaults()
analysis = morph.analyze("yardımcılarınızdan")
for result in analysis:
    print(result.get_stem(), result.get_morphemes())
# Çıktı örneği:
# ya1rdımcı ['Noun', 'Pos', 'Plur', 'A3sg', 'P2pl']

-- **  Ne Zaman Kullanılır? ** -- 

* Türkçe veya Fin-Ural dilleri gibi eklemeli dillerde

* Küçük-orta boy veri setlerinde, kelime bazlı token sayısını düşürmek istediğinizde

* Özellikle bilgi çıkarımı, soru-cevap veya anlamsal analiz gibi görevlerde kök-ek yapısının önemli olduğu durumlarda

## Özetle:

* Morfolojik analiz, clean_text()’ten sonra, Tokenizer’dan önce gelen “ön-işleme” adımıdır.

* Keras’ın Tokenizer sınıfı içinde otomatik bulunmaz; senin kendi pipeline’ında açıkça bu işlemi çağırman gerekir.

* Böylece, tokenizer’a verdiğin her “token” zaten tüm eki atılmış kök veya ek bilgisiyle birlikte birim olarak girer ve vocab’in çok daha temiz, genellemesi güçlü olur.








In [ ]:
import pandas as pd
import unicodedata
import re
import spacy

# === 1) spaCy Türkçe Modelini Kur ve Yükle ===
# Terminal’de bir kez çalıştır: 
#   python -m spacy download tr_core_news_sm
nlp = spacy.load("tr_core_news_sm")

# === 2) Temizleyici Fonksiyon ===
def clean_text(text, lowercase=True, normalize_unicode=True):
    if normalize_unicode:
        text = unicodedata.normalize("NFC", text)
    if lowercase:
        text = text.lower()
    # Türkçe karakterleri koruyup diğerlerini boşlukla değiştir
    text = re.sub(r"[^a-zçğıöşü0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# === 3) Lemmatizasyon Fonksiyonu ===
def lemmatize_text(text):
    """
    spaCy modeliyle verilen metindeki her token'ın lemma (kök) hali.
    """
    doc = nlp(text)
    return " ".join(token.lemma_ for token in doc)

# === 4) Örnek Çalıştırma ===
df = pd.read_csv("örnek_set.csv")
raw_texts = df["input"].astype(str).tolist()

for orig in raw_texts[:5]:
    cleaned   = clean_text(orig)
    lemmatized = lemmatize_text(cleaned)
    print(f"Orijinal:   {orig}")
    print(f"Cleaned:    {cleaned}")
    print(f"Lemmatized: {lemmatized}")
    print("---")


* Gerekli kod aktarımları yukarıda verilmiştir.

---
----

##  Subword / BPE / WordPiece Desteği

### Kavram  
- **Subword tokenization**: Metni tüm kelimeler yerine, sık tekrar eden **alt kelime parçalara** (subword) bölerek işler.  
- **Byte-Pair Encoding (BPE)**: En sık geçen bitişik karakter çiftlerini (byte-pair) iteratif olarak birleştirip sözlüğü oluşturur.  
- **WordPiece**: Google’ın BERT modeli için geliştirdiği, sonraki parça olasılıklarını dikkate alarak alt kelime birimleri seçen yöntem.


### Nasıl Çalışır?  
1. **Başlangıç**: Tüm karakterler veya tek karakter token’larıyla başlar.  
2. **Ölçüm**: Karakter çiftleri veya subword parça frekanslarını hesaplar.  
3. **Birleştirme**: En sık çiftleri veya parçaları birleştirir.  
4. **Tekrar**: İstenilen vocab büyüklüğüne ulaşana dek adım 2–3’ü sürdürür.  
5. **Sonuç**: Hem tam kelimeler, hem de alt parçalar (ör. “yardım” + “cılar”) içeren subword sözlüğü elde edilir.



### Faydaları  
- **OOV Problemini Minimize Eder**  
  - Komple yeni kelimeler bile mevcut alt parçaların birleşimiyle ifade edilir.  
- **Küçük, Etkili Sözlük**  
  - Binlerce nadir kelime yerine, yüzlerce subword birimi yeterli olur.  
- **Çok Dilli Destek**  
  - Birden fazla dilde ortak subword birimleri paylaşarak transfer öğrenmeyi kolaylaştırır.  
- **Hafıza & Hız Kazancı**  
  - Daha küçük embedding matrisleri, daha hızlı eğitim ve çıkarım.



### Ne Zaman Kullanılır?  
- **Büyük veri setleri** ve **çok dilli** uygulamalarda  
- **OOV oranının yüksek** olduğu durumlarda  
- Modelin **daha ince anlamsal** genellemeye ihtiyacı varsa  
- **Transformer** veya **BERT** benzeri subword-temelli modellerde zorunlu



### Uygulama Özet  
- **SentencePiece** (Google): BPE & Unigram modelleri, bağımsız olarak dil ve platformdan  
- **HuggingFace Tokenizers** (Rust tabanlı): Hem BPE hem WordPiece hem Unigram  
- **Tokenizers.fit** → **encode** / **decode** metotlarıyla subword kelimeleri otomatik işle

> Subword tokenization, modern NLP modellerinin temel taşlarından biridir ve OOV sorununu büyük ölçüde ortadan kaldırır.  


### Subword / BPE Tokenization’da “Tokenizer” Nasıl Çalışır?

1. **Klasik `Tokenizer` yerine Subword Model**  
   - Keras’ın `Tokenizer` sınıfını kullanmak yerine, `SentencePiece` (veya HuggingFace Tokenizers) tarafından eğitilmiş bir model kullanırsınız.  
   - Bu model `.Train()` adımıyla bir “vocab.model” dosyası ve “vocab.vocab” dosyası oluşturur.

2. **`fit_on_texts` Gibi Adım Yok**  
   - Metinleri “öğreten” kod, `SentencePieceTrainer.Train(...)` fonksiyonudur; bu fonksiyon ham metin dosyasını (“all_texts.txt”) okur ve BPE sözlüğünü çıkarır.  
   - Dolayısıyla `tokenizer.fit_on_texts(input_texts)` yerine:
     ```python
     spm.SentencePieceTrainer.Train(
         input="all_texts.txt",
         model_prefix="bpe_spm",
         vocab_size=8000,
         model_type="bpe"
     )
     ```
     satırları kullanılır.

3. **Model Yükleme & Kodlama**  
   - Elde edilen `bpe_spm.model` dosyasını:
     ```python
     sp = spm.SentencePieceProcessor()
     sp.Load("bpe_spm.model")
     ```
     ile yüklüyorsunuz.  
   - Bu `sp` nesnesi **tokenizer**’ınız olur.  
   - Ardından metni ID’lere çevirmek için:
     ```python
     ids = sp.EncodeAsIds("merhaba dünya")
     ```
     veya parça bazlı metne döndürmek için
     ```python
     pieces = sp.EncodeAsPieces("merhaba dünya")
     ```

4. **Keras’ın `pad_sequences` ile Entegrasyon**  
   - Kendi `sp` nesnenizle dönüşüm yaptıktan sonra, ID listelerini `pad_sequences(..., value=pad_id)` ile pad edersiniz.  
   - Bu aşamada Keras’a özgü `Tokenizer` kullanımı tamamen devreden çıkar.


**Özet:**  
- Subword tokenization’da **`SentencePiece` veya `Tokenizers` model dosyası** kendi “tokenizer”ınızdır.  
- `fit_on_texts` yerine **`Trainer.Train(...)`** adımını kullanırsınız.  
- Kodlama/çözme işlemleri `sp.EncodeAsIds` ve `sp.EncodeAsPieces` ile yapılır.  
- Sonrasında `pad_sequences` gibi araçlarla Keras pipeline’ınıza bağlarsınız.  


## Kelime-bazlı `Tokenizer` vs. Subword (SentencePiece/BPE)

| Kriter | Keras `Tokenizer.fit_on_texts` <br>(Kelime-bazlı) | SentencePiece / BPE / WordPiece <br>(Subword) |
|--------|---------------------------------------------------|-----------------------------------------------|
| **Kurulum** | Keras içinde hazır, ek paket gerekmez | Ek paket ( `sentencepiece` veya *HF Tokenizers*), model eğitimi gerekir |
| **OOV (Out-of-Vocab) Oranı** | Yüksek — nadir veya yeni kelimeler `<unk>` olur | Çok düşük — yeni kelimeler mevcut alt parçaların birleşimiyle yazılır |
| **Vocab Boyutu** | Kelime başına 1 giriş ⇒ büyük sözlük | Aynı kapsama için çok daha küçük (∼8 k) |
| **Bellek / Embed Matris** | Büyük matris, yavaşlar | Küçük matris, daha hızlı eğitim/çıkarım |
| **Dil Desteği** | İngilizce-benzeri dillerde sorun az | Eklemeli dillerde (TR, FI) özellikle güçlü |
| **İnsan Okunabilirlik** | Çıktı tam kelime; log okumak kolay | Parçalı çıktı (“yardımcı▁lar”), yorumlamak zor |
| **Prototip Hızı** | Çok hızlı başlar, kod sade | Model + dosya üretmek ek adım |
| **Güncelleme Kolaylığı** | `fit_on_texts` tekrar çalıştırmak yeter | Yeni veri için BPE modelini yeniden eğitmek gerekir |
| **Büyük Veri** | Nadir kelimeler ve OOV patlar | Kapsama hâlâ yüksek, ölçeklenebilir |
| **Standart Modern Modeller** | LSTM/GRU tabanlı seq2seq | BERT, T5, GPT, Transformer standardı |



### Ne Zaman Hangi Yöntem?

- **Hızlı prototip / küçük veri**  
  - *Kelime-bazlı Tokenizer*  
  - Daha okunaklı log, daha az dış bağımlılık.

- **Veri > ~1 M cümle, OOV % ↑, çok dilli veya eklemeli dil**  
  - *SentencePiece (BPE / WordPiece)*  
  - Daha küçük vocab, neredeyse sıfır OOV, modern Transformer’larla tam uyum.

- **Ara Çözümler**  
  - Kelime-bazlı Tokenizer + **lemmatizasyon & `min_freq`**  
  - OOV hâlâ yüksekse → **Subword**’a geç.

> **Pratik Yol Haritası**  
> 1. Kelime-bazlı ile başla.  
> 2. OOV % > 5–10 ise lemmatizasyon/min_freq uygula.  
> 3. Hâlâ sorunluysa veya veri büyüyorsa SentencePiece’e geç.


In [1]:
import os, io, sentencepiece as spm
import pandas as pd
import unicodedata, re, numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv("örnek_set.csv")

raw_inputs = df["input"].astype(str).to_list()
raw_targets = df["output"].astype(str).to_list()

def clean_texts(text:str) -> str:
    text = unicodedata.normalize("NFC",text).lower()
    text = re.sub(r"[^a-zçğıöşü0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

input_texts = [clean_texts(t)  for t in raw_inputs]
target_texts = [clean_texts(t) for t in raw_targets]

# ------------------------------------
# 2) BPE modelini SentencePiece ile eğit
# ------------------------------------

model_prefix = "bpe_tr"
vocab_size = 8000
data_txt = "corpus.txt"

with io.open(data_txt,"w",encoding = "utf-8") as f:
    for t in input_texts + target_texts:
        f.write(t + "\n")

# 2) BPE Modelini eğitirken:
UNIQUE_LIMIT = 1800          # Trainer’ın verdiği üst sınırın biraz altı
spm.SentencePieceTrainer.Train(
    input=data_txt,
    model_prefix=model_prefix,
    model_type="bpe",
    vocab_size=min(vocab_size, UNIQUE_LIMIT),   # 8k yerine 1 800
    user_defined_symbols=["<pad>", "<sos>", "<eos>"]
)

# -----------------------------
# 3) Modeli yükle & özel ID’ler
# -----------------------------

sp = spm.SentencePieceProcessor(model_file=f"{model_prefix}.model")
pad_id = sp.piece_to_id("<pad>")
sos_id = sp.piece_to_id("<sos>")
eos_id = sp.piece_to_id("<eos>")


# ---------------------------------
# 4) Metinleri ID listesine dönüştür
# ---------------------------------
enc_seqs = [sp.encode(t, out_type=int) for t in input_texts]
dec_in   = [[sos_id] + sp.encode(t, out_type=int)         for t in target_texts]
dec_out  = [            sp.encode(t, out_type=int) + [eos_id] for t in target_texts]


# --------------  
# 5) Padding
# --------------
max_len = max(max(map(len, enc_seqs)),
              max(map(len, dec_in)),
              max(map(len, dec_out)))

print(max_len)

encoder_input  = pad_sequences(enc_seqs, maxlen=max_len, padding="post", value=pad_id)
decoder_input  = pad_sequences(dec_in,   maxlen=max_len, padding="post", value=pad_id)
decoder_target = pad_sequences(dec_out,  maxlen=max_len, padding="post", value=pad_id)

12


In [5]:

# -----------------------------------
# 6) tf.data.Dataset pipeline
# -----------------------------------
BATCH_SIZE  = 32
BUFFER_SIZE = 1000

dataset = tf.data.Dataset.from_tensor_slices(
    (
        {"encoder_input": encoder_input,
         "decoder_input": decoder_input},
        decoder_target
    )
).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

print("Pipeline hazır ➜ dataset örneği:", next(iter(dataset.take(1)))[0]["encoder_input"].shape)
print("pad_id:", pad_id, "sos_id:", sos_id, "eos_id:", eos_id)
print("max_len:", max_len, "vocab_size:", vocab_size)

Pipeline hazır ➜ dataset örneği: (32, 12)
pad_id: 3 sos_id: 4 eos_id: 5
max_len: 12 vocab_size: 8000


-----
-----

## Chatbot-Odaklı Transformer İçin Ek Tokenizer / Veri İyileştirmeleri

| Başlık | Neden Önemli? | Nasıl Eklenir? |
|--------|---------------|----------------|
| **Konuşmacı Etiketleri** (`<user>`, `<bot>`) | Rol ayrımı, diyaloğun yönünü netleştirir; model hangi repliğin kime ait olduğunu öğrenir. | Her cümleye başlık token’ı ekleyin: `" <user> merhaba"` / `" <bot> merhaba, nasıl yardımcı olabilirim?"` |
| **Diyalog Tur Ayırıcı** (`<turn>`) | Uzun geçmişte cümle sınırlarını netleştirir; kontekst “kaymasını” azaltır. | Çok-cümleli geçmişi tek dizide birleştirip her cümle sonuna `<turn>` ekleyin. |
| **Persona / Stil Token’ları** (`<formal>`, `<casual>`) | Kişiselleştirme ve ton kontrolü; aynı modelle birden fazla stil üretimi. | Eğitim verisinde örnek yanıtın başına uygun token ekleyin. |
| **Subword Regularization** (BPE sampling) | Çeşitli alt-parça varyasyonları → overfitting düşer, robustluk artar. | `SentencePieceProcessor.Encode(..., enable_sampling=True, nbest_size=-1, alpha=0.1)` |
| **Dynamic Padding Buckets** | Sabit `max_len` yerine yakın uzunlukta cümleleri gruplayıp %30-40 GPU kazanımı. | `tf.data.experimental.bucket_by_sequence_length()` kullanın. |
| **Numerik / Tarih Placeholder’ları** (`<NUM>`, `<DATE>`) | Rakam kümeleri anlamsal gürültü yaratır; generatif hataları azaltır. | Regex ile sayıları `<NUM>123</NUM>` veya sadece `<NUM>` şeklinde maskeleyin. |
| **NER-tabanlı De-lexicalization** (`<PERSON>`, `<ORG>`) | Gizlilik + daha iyi genelleme; özel isimler OOV yaratmaz. | SpaCy / Stanza NER → özel isimleri placeholder ile değiştir. |
| **Token Dropout Augmentation** | Beklenmedik eksik kelimelere karşı dayanıklılık. | `dataset.map(token_dropout, num_parallel_calls=AUTOTUNE)` |
| **Segment ID veya Speaker ID Embedding’i** | Self-Attention’ın “kim konuşuyor” bilgisini doğrudan görmesi. | Transformer girdi dict’ine `speaker_ids` tensörü ekleyin (0=user,1=bot). |
| **Label Smoothing** | Hedef dizideki tek-sıcak (one-hot) köşeliğini yumuşatır, over-confidence önler. | `tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)` |
| **Top-K / Nucleus Decoding Ayar Token’ları** (`<topk_50>`, `<topp_0.9>`) | Çıkarımda esnek kontrol: tutarlılık vs. yaratıcılık. | Prompt’un başına ayar token’ı koy, decode sırasında parse edip karşılık gelen sampling config’i kullan. |


* Şimdi aşağıya artık son demlerine gelmiş bir tokenizer kodu bırakıyorum ( Subword Tokenization – SentencePiece ile Byte-Pair Encoding (BPE ).Bu diğer tokenizer işlemi.Bundan sonra ise ilk başlarda uyguladığımız tokenizer işlemlerine eklemeler yapacağız.

In [7]:
"""
SUBWORD-TABANLI TOKENIZER PIPELINE  (masking yok)

● SentencePiece-BPE modeli (1 800 parçalık sözlük)
● <sos>/<eos>/<pad>/<unk> özel token’ları
● Rastgele %10 token’ı <unk> yaparak UNK-dropout (data augmentation)
● tf.data.Dataset → shuffle-batch-prefetch zinciri
"""

# ------------------------ 0. Gereksinimler ------------------------
# pip install sentencepiece tensorflow==2.18 pandas

import io, re, unicodedata, sentencepiece as spm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ------------------------ 1. Veriyi Yükle & Temizle ---------------
df = pd.read_csv("örnek_set.csv")           # input ve output sütunları
raw_inputs  = df["input"].astype(str).tolist()
raw_targets = df["output"].astype(str).tolist()

def clean(text: str) -> str:
    text = unicodedata.normalize("NFC", text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

input_texts  = [clean(t) for t in raw_inputs]
target_texts = [clean(t) for t in raw_targets]

# ------------------------ 2. SentencePiece BPE Eğit ----------------
MODEL_PREFIX = "bpe_tr"
VOCAB_SIZE   = 8000                 # istenen
UNIQUE_LIMIT = 1800                 # küçük korpus üst limiti
CORPUS_TXT   = "corpus.txt"

with io.open(CORPUS_TXT, "w", encoding="utf-8") as f:
    for t in input_texts + target_texts:
        f.write(t + "\n")

spm.SentencePieceTrainer.Train(
    input=CORPUS_TXT,
    model_prefix=MODEL_PREFIX,
    model_type="bpe",
    vocab_size=min(VOCAB_SIZE, UNIQUE_LIMIT),
    user_defined_symbols=["<pad>", "<sos>", "<eos>"]
)

# ------------------------ 3. Modeli Yükle & ID’ler ----------------
sp = spm.SentencePieceProcessor(model_file=f"{MODEL_PREFIX}.model")
pad_id = sp.piece_to_id("<pad>")
sos_id = sp.piece_to_id("<sos>")
eos_id = sp.piece_to_id("<eos>")
unk_id = sp.piece_to_id("<unk>")
vocab_size = sp.get_piece_size()

# ------------------------ 4. Encode & Padding ---------------------
enc_seqs = [sp.encode(t, out_type=int) for t in input_texts]
dec_in   = [[sos_id] + sp.encode(t, out_type=int) for t in target_texts]
dec_out  = [sp.encode(t, out_type=int) + [eos_id] for t in target_texts]

max_len = max(
    max(map(len, enc_seqs)),
    max(map(len, dec_in)),
    max(map(len, dec_out))
)

encoder_input  = pad_sequences(enc_seqs, maxlen=max_len, padding="post", value=pad_id)
decoder_input  = pad_sequences(dec_in,   maxlen=max_len, padding="post", value=pad_id)
decoder_target = pad_sequences(dec_out,  maxlen=max_len, padding="post", value=pad_id)

# ------------------------ 5. UNK-Dropout Fonksiyonu ---------------
DROPOUT_RATE = 0.10   # %10 token rastgele <unk>

def unk_dropout(inputs, targets, rate=DROPOUT_RATE, unk=unk_id):
    enc = inputs["encoder_input"]
    dec = inputs["decoder_input"]

    mask_enc = tf.cast(tf.random.uniform(tf.shape(enc)) < rate, enc.dtype)
    mask_dec = tf.cast(tf.random.uniform(tf.shape(dec)) < rate, dec.dtype)

    inputs["encoder_input"] = tf.where(mask_enc == 1, unk, enc)
    inputs["decoder_input"] = tf.where(mask_dec == 1, unk, dec)
    return inputs, targets

# ------------------------ 6. Dataset Pipeline ---------------------
BATCH_SIZE  = 32
BUFFER_SIZE = 1000

dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            {"encoder_input": encoder_input,
             "decoder_input": decoder_input},
            decoder_target
        )
    )
    .map(unk_dropout, num_parallel_calls=tf.data.AUTOTUNE)   # sadece UNK-dropout
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

# ------------------------ 7. Kontrol Çıktıları --------------------
enc_ex = next(iter(dataset.take(1)))[0]["encoder_input"]
print("dataset örneği encoder shape:", enc_ex.shape)
print(f"pad_id:{pad_id}  sos_id:{sos_id}  eos_id:{eos_id}  unk_id:{unk_id}")
print("max_len:", max_len, "   vocab_size:", vocab_size)


dataset örneği encoder shape: (32, 12)
pad_id:3  sos_id:4  eos_id:5  unk_id:0
max_len: 12    vocab_size: 1800


* Son olarak da "<user" ve "<bot" taglarını ekleyip işi bırakalım.

In [8]:
"""
TOKENIZER PIPELINE + KONUŞMACI & STİL TOKEN’LARI
-------------------------------------------------
• SentencePiece-BPE (≈1 800 parçalık sözlük)
• Özel semboller: <pad> <sos> <eos> <user> <bot> <formal> <casual>
• UNK-Dropout (%10)  →  dayanıklı girdi
• tf.data.Dataset  →  shuffle • batch • prefetch
(Attention mask EKLENMEDİ — yalnızca tokenizer & pipeline)
"""

# --------------------- 0. Gereksinimler ---------------------
# pip install sentencepiece pandas tensorflow==2.18

import io, re, unicodedata, sentencepiece as spm
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --------------------- 1. Veriyi Yükle & Temizle ------------
df = pd.read_csv("örnek_set.csv")
raw_inputs  = df["input"].astype(str).tolist()
raw_targets = df["output"].astype(str).tolist()

def clean(text: str) -> str:
    text = unicodedata.normalize("NFC", text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

# Konuşmacı + stil etiketleri ekle
SPEAKER_USER = "<user>"
SPEAKER_BOT  = "<bot>"
STYLE_CASUAL = "<casual>"

input_texts  = [f"{SPEAKER_USER} {STYLE_CASUAL} {clean(t)}" for t in raw_inputs]
target_texts = [f"{SPEAKER_BOT} {STYLE_CASUAL} {clean(t)}" for t in raw_targets]

# --------------------- 2. SentencePiece BPE Eğit ------------
MODEL_PREFIX = "bpe_tr"
VOCAB_SIZE   = 8000
LIMIT        = 1800                     # küçük korpus sınırı
CORPUS_TXT   = "corpus.txt"

with io.open(CORPUS_TXT, "w", encoding="utf-8") as f:
    for t in input_texts + target_texts:
        f.write(t + "\n")

spm.SentencePieceTrainer.Train(
    input=CORPUS_TXT,
    model_prefix=MODEL_PREFIX,
    model_type="bpe",
    vocab_size=min(VOCAB_SIZE, LIMIT),
    user_defined_symbols=[
        "<pad>", "<sos>", "<eos>",
        SPEAKER_USER, SPEAKER_BOT, STYLE_CASUAL
    ]
)

sp = spm.SentencePieceProcessor(model_file=f"{MODEL_PREFIX}.model")

pad_id = sp.piece_to_id("<pad>")
sos_id = sp.piece_to_id("<sos>")
eos_id = sp.piece_to_id("<eos>")
unk_id = sp.unk_id()
vocab_size = sp.get_piece_size()

# --------------------- 3. Encode & Pad ----------------------
enc_seqs = [sp.encode(t, out_type=int) for t in input_texts]
dec_in   = [[sos_id] + sp.encode(t, out_type=int) for t in target_texts]
dec_out  = [sp.encode(t, out_type=int) + [eos_id] for t in target_texts]

max_len = max(
    max(map(len, enc_seqs)),
    max(map(len, dec_in)),
    max(map(len, dec_out))
)

encoder_input  = pad_sequences(enc_seqs, maxlen=max_len, padding="post", value=pad_id)
decoder_input  = pad_sequences(dec_in,   maxlen=max_len, padding="post", value=pad_id)
decoder_target = pad_sequences(dec_out,  maxlen=max_len, padding="post", value=pad_id)

# --------------------- 4. UNK-Dropout -----------------------
DROPOUT_RATE = 0.10   # %10 token rastgele <unk>

def unk_dropout(inputs, targets, rate=DROPOUT_RATE, unk=unk_id):
    enc = inputs["encoder_input"]
    dec = inputs["decoder_input"]
    mask_enc = tf.cast(tf.random.uniform(tf.shape(enc)) < rate, enc.dtype)
    mask_dec = tf.cast(tf.random.uniform(tf.shape(dec)) < rate, dec.dtype)
    inputs["encoder_input"] = tf.where(mask_enc == 1, unk, enc)
    inputs["decoder_input"] = tf.where(mask_dec == 1, unk, dec)
    return inputs, targets

# --------------------- 5. Dataset Pipeline ------------------
BATCH_SIZE  = 32
BUFFER_SIZE = 1000

dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            {"encoder_input": encoder_input,
             "decoder_input": decoder_input},
            decoder_target
        )
    )
    .map(unk_dropout, num_parallel_calls=tf.data.AUTOTUNE)  # sadece tokenizer augmentation
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

# --------------------- 6. Kontrol Çıktıları -----------------
enc_example = next(iter(dataset.take(1)))[0]["encoder_input"]
print("Encoder batch shape:", enc_example.shape)
print(f"pad_id:{pad_id}  sos_id:{sos_id}  eos_id:{eos_id}  unk_id:{unk_id}")
print("max_len:", max_len, "   vocab_size:", vocab_size)


Encoder batch shape: (32, 16)
pad_id:3  sos_id:4  eos_id:5  unk_id:0
max_len: 16    vocab_size: 1800


----
----

## Mevcut Kelime-Bazlı Tokenizer Kodunu “Derinleştirme” Yol Haritası

>Amaç: **BPE’yi zaten çözdük**; şimdi klasik `Tokenizer` senaryosunu daha esnek, yeniden-kullanılabilir ve “üretim hazır” hâle getirmek.


### 1. Kodun Fonksiyonlaştırılması
Tek seferlik betik yerine **parametrik fonksiyon** yazmak:
```python
def build_word_tokenizer(
        input_texts,
        target_texts,
        min_freq    = 2,
        max_vocab   = None,
        filters     = "",
        oov_token   = "<unk>",
        pad_token   = "<pad>",
        sos_token   = "<sos>",
        eos_token   = "<eos>",
        char_level  = False,
        padding     = "post"):

    # (kodun 1-8. adımları burada özetlenmiş hâlde)
    return tokenizer, encoder_input, decoder_input, decoder_target


### Konuşmacı & Stil Etiketlerini Kelime-Bazlı Tokenizer’a Ekleme

#### 1. Neden Gerekli?
| Sorun | Etiketlerin Çözümü |
|-------|-------------------|
| **Rol Belirsizliği** – Model “kimin” konuştuğunu ayırt edemez. | `<user>` ve `<bot>` token’ları konuşmacı rolünü açıkça belirtir. |
| **Ton Kontrolü** – Aynı modelle resmî / samimî yanıt üretmek. | `<casual>` / `<formal>` gibi stil token’ları istenen tonu sinyaller. |
| **Kontekst Kayması** – Uzun diyaloğun nerede “bölündüğü” anlaşılmaz. | Etiketler satır başında sabit kalıp kontekst hizalamasını korur. |

#### 2. Nasıl Çalışır?
1. **Ön-işleme**:  
   Her ham cümlenin başına **rol** ve **stil** token’ı eklenir:  
   ```text
   <user> <casual> merhaba
   <bot>  <casual> merhaba, nasıl yardımcı olabilirim?


In [6]:
df = pd.read_csv(r"C:\Users\hdgn5\OneDrive\Masaüstü\Transformerlar\Konu Anlatımları\Encoder - Decoder - PE - ATTN ( İLERİ DÜZEY ) = TF\Tokenizer\örnek_set .csv")
raw_inputs  = df["input"].astype(str).tolist()
raw_targets = df["output"].astype(str).tolist()

In [9]:
def clean(text: str) -> str:
    text = unicodedata.normalize("NFC", text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()


* Aşağıda bulunan 2 satır bu işlemi anlatıyor.

In [10]:
USER , BOT ,CASUAL = "<user>" , "<bot>" , "<casual>"

In [12]:
input_texts = [f"{USER} {CASUAL} {clean(t)}" for t in raw_inputs]
target_texts = [f"{BOT} {CASUAL} {clean(t)}"  for t in raw_targets]

* Diğer adımlar aynı şekilde işleme devam eder.

## Dynamic UNK-Dropout (Kelime-bazlı Tokenizer’a Zarar Vermeden)

### Neden Dinamik?
* **Sabit oran** (ör. 0.10) her epoch boyunca aynı gürültüyü verir.  
* **Dinamik takvim**:  
  - **Erken epoch’larda** düşük dropout ⇒ model temel yapıyı öğrenir.  
  - **Orta epoch’larda** yüksek dropout ⇒ genelleme güçlenir.  
  - **Sonlara doğru** oranı tekrar düşür ⇒ duyarlılık kazanılır.


### Basit Takvim Örneği  
| Epoch Aralığı | Oran (`rate`) | Yorum |
|---------------|--------------|-------|
| 0 – 4         | 0.05         | “Isınma” – saf veriye yakın |
| 5 – 14        | 0.12         | “Sağlamlaştırma” |
| 15 – 19       | 0.08         | “İnce ayar” |


In [13]:
"""
DİNAMİK UNK-DROPOUT + KONUŞMACI/STİL ETİKETLİ KELİME-BAZLI TOKENIZER
────────────────────────────────────────────────────────────────────

• <user> / <bot> + <casual> etiketleri
• min_freq filtresi (nadir kelimeleri OOV’a at)
• Dynamic UNK-Dropout takvimi:
      epoch 0-4  →  0.05
      epoch 5-14 →  0.12
      epoch 15-19→  0.08
• tf.data.Dataset → shuffle • batch • prefetch
(Masking/model kısmı bu betiğe dâhil değil)
"""

'\nDİNAMİK UNK-DROPOUT + KONUŞMACI/STİL ETİKETLİ KELİME-BAZLI TOKENIZER\n────────────────────────────────────────────────────────────────────\n\n• <user> / <bot> + <casual> etiketleri\n• min_freq filtresi (nadir kelimeleri OOV’a at)\n• Dynamic UNK-Dropout takvimi:\n      epoch 0-4  →  0.05\n      epoch 5-14 →  0.12\n      epoch 15-19→  0.08\n• tf.data.Dataset → shuffle • batch • prefetch\n(Masking/model kısmı bu betiğe dâhil değil)\n'

In [14]:

# -----------------------------------------------------------------
# 0) Gereksinimler
#    pip install pandas tensorflow==2.18
# -----------------------------------------------------------------
import re, unicodedata, json, pandas as pd, tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [79]:
filters        = ""            # Noktalama işleme filtresi
oov_token      = "<unk>"       # Out-of-vocab token
pad_token      = '<pad>'       # Padding token
sos_token      = "<sos>"       # Decoder başlangıç token
eos_token      = "<eos>"       # Decoder bitiş token
char_level     = False         # True = karakter, False = kelime
max_vocab_size = None          # None = sınırsız
min_freq       = 2             # En az 2 kez geçen token'lar tutulacak
padding       = "post"         # "pre" da seçilebilir

# === 2. Sabit Token Tanımları ===
USER, BOT, CASUAL = "<user>", "<bot>", "<casual>"

fixed_index = {
    pad_token : 0,
    oov_token : 1,
    sos_token : 2,
    eos_token : 3
}

In [80]:
def clean_text(text:str)->str:
    text = unicodedata.normalize("NFC",text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

In [81]:
input_texts = [f"{USER} {CASUAL} {clean_text(t)}" for t in raw_inputs]
target_texts = [f"{BOT} {CASUAL} {clean_text(t)}" for t in raw_targets]

In [82]:
# === 4. SOS/EOS Ekleme ===
dec_in_text  = [f"{sos_token} {t}" for t in target_texts]
dec_out_text = [f"{t} {eos_token}" for t in target_texts]
all_texts    = input_texts + dec_in_text + dec_out_text

In [83]:
# --------------------------- 4. Geçici Tokenizer -----------------
tmp_tok = Tokenizer(filters=filters, oov_token=oov_token, char_level=char_level)
tmp_tok.fit_on_texts(all_texts)
word_counts = tmp_tok.word_counts

In [84]:
keep = [w for w, c in word_counts.items() if c >= min_freq] # min_freq uugulaması.
for special in (USER,BOT,CASUAL,pad_token,sos_token,eos_token):
    if special not in keep:
        keep_tokens.append(special)

In [85]:
tokenizer = Tokenizer(num_words=len(keep) + 1, filters=filters,oov_token=oov_token ,char_level=char_level)
tokenizer.fit_on_texts(keep)
tokenizer.fit_on_texts(all_texts)

In [86]:
inp_seq = tokenizer.texts_to_sequences(input_texts)
dec_in_seq = tokenizer.texts_to_sequences(dec_in_text)
dec_in_out = tokenizer.texts_to_sequences(dec_out_text)

In [87]:
maxlen = max(map(len, inp_seq+dec_in_seq+dec_in_out))
print(maxlen)

14


In [92]:
enc_in  = pad_sequences(inp_seq,    maxlen=maxlen, padding=padding, value=0)
dec_in  = pad_sequences(dec_in_seq, maxlen=maxlen, padding=padding, value=0)
dec_out = pad_sequences(dec_in_out,maxlen=maxlen, padding=padding, value=0)

# --- Dinamik UNK-dropout için değişken tanımı ---

In [94]:
unk_id = tokenizer.word_index[oov_token]
drop_var = tf.Variable(0.05 , trainable=False,dtype=tf.float32)

def unk_dropout_dyn(inputs,targets):

    e = inputs["encoder_input"] ; d = inputs["decoder_input"]
    m_e = tf.cast(tf.random.uniform(tf.shape(e)) < drop_var , e.dtype)
    m_d = tf.cast(tf.random.uniform(tf.shape(d)) < drop_var , d.dtype)

    inputs["encoder_input"] = tf.where(m_e==1 , unk_id, e)
    inputs["decoder_input"] = tf.where(m_d==1 , unk_id , d)

    return inputs , targets

In [95]:
batch , BUFFER = 32, 1000
dataset = (
    tf.data.Dataset.from_tensor_slices((
        {"encoder_input" : enc_in , "decoder_input" : dec_in} , dec_out
        ))
        .map(unk_dropout_dyn , num_parallel_calls=tf.data.AUTOTUNE)
        .shuffle(BUFFER)
        .batch(batch,drop_remainder=True)
        .prefetch(tf.data.AUTOTUNE)
    )

In [97]:
# --- Callback ile oran takvimi ---

def schedule(ep):

    if ep < 5: return 0.05
    if ep < 15 : return 0.12

    return 0.08

class UnkScheduler(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs=None):
        new = schedule(epoch)
        drop_var.assign(new)
        print(f"\n[UNK-Dropout] epoch {epoch} → rate={new:.2f}")

#####  Artık model.fit(dataset, epochs=20, callbacks=[UnkScheduler()]) diyebilirsin.

----
----

# Düz Tokenizer vs. Subword-BPE Pipeline: Hangisi Daha Profesyonel ve Kullanışlı?

Aşağıda “1) Düz kelime-bazlı tokenizer + Dynamic UNK-Dropout + speaker/stil token’ları” ile “2) SentencePiece-BPE tabanlı tokenizer + UNK-Dropout” yaklaşımlarını karşılaştırdım. Projenizin ölçeği, hedefleri ve ekosistemi doğrultusunda hangisinin size daha uygun olduğuna karar verebilirsiniz.

| Özellik                          | 1) Düz Kelime-bazlı Tokenizer                       | 2) Subword-BPE (SentencePiece)                         |
|----------------------------------|------------------------------------------------------|---------------------------------------------------------|
| **Kurulum / Bağımlılıklar**      | Sadece `tensorflow`/`keras`                           | + `sentencepiece` paketi                                |
| **Sözlük Oluşumu**               | `Tokenizer.fit_on_texts` + `min_freq` + `max_vocab`  | `SentencePieceTrainer.Train` (dış model dosyası)        |
| **OOV (Bilinmeyen Token)**       | Yüksek (%OOV ≈ 100−coverage)                         | Pratikte sıfıra yakın (her yeni kelime parçalara bölünür) |
| **Vocab Boyutu**                 | Kelime sayısına göre çok büyük, `min_freq` şart      | Küçük ve kontrollü (örn. 1 000–8 000 parça)             |
| **Bellek & Performans**          | Büyük embed matrisi → daha fazla bellek              | Küçük embed matrisi → hem eğitim hem çıkarım hızlanır  |
| **Dil Yapısı Uyum**              | Eklemeli dillerde (Türkçe) çok nadir token’lı        | Türkçe gibi eklemeli dillerde doğal subword ayrışımı     |
| **Konuşmacı/Stil Token’ları**    | Her iki yaklaşmada kolayca eklenir                   | Aynı etiketler `user_defined_symbols` ile eklenir       |
| **Dynamic UNK-Dropout**          | Callback + `tf.data.map` içinde uygulamak kolay      | Yine `tf.data.map` ile direkt entegre                  |
| **Pipeline Karmaşıklığı**        | Daha sade, tek kod bloğu                             | Biraz daha uzun; model eğitimi + encode + pad adımları |
| **Model Uyumluluğu**             | LSTM/GRU veya basit seq2seq                          | Modern Transformer / BERT/GPT tabanlı mimarilerle birebir uyum |
| **Prototip & Ölçek**             | Küçük projeler, hızlı deneme                         | Büyük veri, çok dilli veya üretim odaklı               |



## Ne Zaman Hangisi?

- **Hızlı Prototip / Küçük Veri**  
  Düz kelime-bazlı tokenizer:
  - Dış paket kurmadan, birkaç satırlık kodla hazır.
  - İnsan okunurluğu yüksek, hata ayıklamak kolay.

- **Büyük Veri / Üretim / Transformer-Uyumluluk**  
  Subword-BPE:
  - OOV sorununu pratikte ortadan kaldırır.
  - Sözlük boyutunu kontrol altında tutar.
  - Modern NLP modelleriyle tam uyumlu.


## Sonuç ve Öneri

- Eğer **Türkçe chatbot** projeniz 10 k–100 k cümle civarında, dış bağımlılıklardan kaçınmak istiyorsanız, **düz tokenizer + dynamic UNK-dropout** gayet yeterli olacaktır.
- Eğer **veri setiniz milyonlarca** cümleye ulaşıyor, OOV’ları sıfıra yakın görmek ve **Transformer**/**GPT-like** mimarilerle üretim kalitesi hedefliyorsanız, **SentencePiece-BPE** çözümünü tercih edin.

Her iki pipeline da konuşmacı/stil token’ları ve dinamik augmentasyon adımlarını destekler. Projeye ilk düz tokenizer’la başlayıp, ölçek ve ihtiyaç arttıkça BPE’ye geçiş yapmak, en az israfla en hızlı sonuç almanıza yardımcı olur.  
